# Loading Passive Data

In [ ]:
# Turn off logging and disable tqdm this is a hidden cell on docs page
import logging
import os

logger = logging.getLogger('ibllib')
logger.setLevel(logging.CRITICAL)

os.environ["TQDM_DISABLE"] = "1"

Passive stimuli related events. The passive protocol is split into three sections
1. Spontaneous activity (SP)
2. Receptive Field Mapping (RFM)
3. Task replay (TR)

## Relevant datasets
* passivePeriods.intervalsTable.csv (SP)
* passiveRFM.times.npy (RFM)
* \_iblrig_RFMapStim.raw.bin (RFM)
* passiveGabor.table.csv (TR - visual)
* passiveStims.table.csv (TR - auditory)


## Loading

### Loading spontaneous activity

In [ ]:
from one.api import ONE
one = ONE()
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'

passive_times = one.load_dataset(eid, '*passivePeriods*', collection='alf')
SP_times = passive_times['spontaneousActivity']

### Loading recpetive field mapping

In [ ]:
from brainbox.io.one import load_passive_rfmap

RFMap = load_passive_rfmap(eid, one=one)

### Loading task replay

In [ ]:
# Load visual stimulus task replay events
visual_TR = one.load_dataset(eid, '*passiveGabor*', collection='alf')

# Load auditory stimulus task replay events
auditory_TR = one.load_dataset(eid, '*passiveStims*', collection='alf')

## More details
* [Description of passive datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.81i06nkedtbe)
* [Decsription of passive protocol](https://docs.google.com/document/d/1PkN_-jWXBLAWbONWXVa2JZh3D9tfurNGsXh422dUxMo/edit#heading=h.fiffmd82uci7)

## Useful modules
* [brainbox.io.one](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.io.one.html#brainbox.io.one.load_passive_rfmap)
* [brainbox.task.passive](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.task.passive.html)
* [ibllib.io.extractors.extract_passive](https://int-brain-lab.github.io/iblenv/_autosummary/ibllib.io.extractors.ephys_passive.html#module-ibllib.io.extractors.ephys_passive)

## Exploring passive data

### Example 1: Compute firing rate for each cluster during spontaneous activity

In [ ]:
# Find first probe insertion for session
pid = one.alyx.rest('insertions', 'list', session=eid)[0]['id']

from brainbox.io.one import SpikeSortingLoader
from iblatlas.atlas import AllenAtlas
import numpy as np
ba = AllenAtlas()

# Load in spikesorting
sl = SpikeSortingLoader(pid=pid, one=one, atlas=ba)
spikes, clusters, channels = sl.load_spike_sorting()
clusters = sl.merge_clusters(spikes, clusters, channels)

# Find spike times during spontaneous activity
SP_idx = np.bitwise_and(spikes['times'] >= SP_times[0], spikes['times'] <= SP_times[1])

# Count the number of clusters during SP time period and compute firing rate
from brainbox.population.decode import get_spike_counts_in_bins
counts, cluster_ids = get_spike_counts_in_bins(spikes['times'][SP_idx], spikes['clusters'][SP_idx], 
                                               np.c_[SP_times[0], SP_times[1]])
fr = counts / (SP_times[1] - SP_times[0])

### Example 2: Find RFM stimulus positions and timepoints

In [ ]:
# Find out at what times each voxel on the screen was turned 'on' (grey to white) or turned 'off' (grey to black)
from brainbox.task.passive import get_on_off_times_and_positions

RF_frame_times, RF_frame_pos, RF_frame_stim = get_on_off_times_and_positions(RFMap)

# Find times where pixel at location x=1, y=4 on display was turned 'on'
pixel_idx = np.bitwise_and(RF_frame_pos[:, 0] == 1, RF_frame_pos[:, 1] == 4)
stim_on_frames = RF_frame_stim['on'][pixel_idx]
stim_on_times = RF_frame_times[stim_on_frames[0][0]]

## Other relevant examples
* COMING SOON